In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import tensorflow as tf
import tensorflow.keras.layers as tfl
import tempfile
from PIL import Image
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation
from keras.applications.mobilenet_v2 import MobileNetV2

In [ ]:
!pip install tensorflow_model_optimization
!pip install keras-efficientnets
!pip install keras.utils
!pip install keras_applications

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 238 kB 27.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for keras.utils: filename=keras_utils-1.0.13-py3-none-any.whl size=2656 sha256=51b9e5b2d92884c38dd9ffd4908818ba45b51452d99deccb7cbf32736ea3eb45
  Stored in directory: /root/.cache/pip/wheels/d0/dd/3b/493952a5240d486a83805d65360dedadbadeae71d25e2c877f
Successfully built keras.utils
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 7.0 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import os

import cv2
import shutil

#from tqdm import tqdm
# tqdm doesn't work well in colab.
# This is the solution:
# https://stackoverflow.com/questions/41707229/tqdm-printing-to-newline
import tqdm.notebook as tq
#for i in tq.tqdm(...):

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#@title
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title
os.listdir('/content/drive/MyDrive/cifar-100-python')

In [ ]:
#@title
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
#@title
path = '/content/drive/MyDrive/cifar-100-python/train'

train_dict = unpickle(path)

train_dict.keys()

In [ ]:
#@title
path = '/content/drive/MyDrive/cifar-100-python/meta'

names_dict = unpickle(path)

names_dict.keys()

In [ ]:
#@title
fine_labels_list = train_dict[b'fine_labels']
coarse_labels_list = train_dict[b'coarse_labels']

fine_label_names_list = names_dict[b'fine_label_names']
coarse_label_names_list = names_dict[b'coarse_label_names']

print(len(fine_labels_list))
print(len(coarse_labels_list))
print(len(fine_label_names_list))
print(len(coarse_label_names_list))

In [ ]:
#@title
# Create the df_train dataframe
df_train = pd.DataFrame(fine_labels_list, columns=['fine_labels'])

# Create new columns
df_train['coarse_labels'] = coarse_labels_list
df_train['image_num'] = df_train.index + 100000

# Create the image_id column
def create_imageid(row):
    
    image_id = str(row['fine_labels']) + '_' + str(row['coarse_labels']) + '_' + str(row['image_num']) +'.jpg'
    
    return image_id

df_train['image_id'] = df_train.apply(create_imageid, axis=1)


# Create the fine and coarse label names columns

def create_finelabelname(x):
    
    # this returns bytes: b'apple'
    name = fine_label_names_list[x]
    
    # convert bytes to string: 'apple'
    name = name.decode("utf-8") 
    
    return name


def create_coarselabelname(x):
    
    # this returns bytes: b'apple'
    name = coarse_label_names_list[x]
    
    # convert bytes to string: 'apple'
    name = name.decode("utf-8") 
    
    return name


df_train['fine_label_names'] = df_train['fine_labels'].apply(create_finelabelname)

df_train['coarse_label_names'] = df_train['coarse_labels'].apply(create_coarselabelname)


# Remove unnecessary columns
df_train = df_train.drop('image_num', axis=1)

# Reorder the columns
cols = ['image_id', 'fine_label_names', 'fine_labels', 'coarse_label_names', 'coarse_labels']
df_train = df_train[cols]

df_train.head()

In [ ]:
#@title
path = '/content/drive/MyDrive/cifar-100-python/test'

test_dict = unpickle(path)

test_dict.keys()

In [ ]:
#@title
fine_labels_list = test_dict[b'fine_labels']
coarse_labels_list = test_dict[b'coarse_labels']

fine_label_names_list = names_dict[b'fine_label_names']
coarse_label_names_list = names_dict[b'coarse_label_names']

print(len(fine_labels_list))
print(len(coarse_labels_list))
print(len(fine_label_names_list))
print(len(coarse_label_names_list))

In [ ]:
#@title
# Create the df_test dataframe
df_test = pd.DataFrame(fine_labels_list, columns=['fine_labels'])

# Create new columns
df_test['coarse_labels'] = coarse_labels_list
df_test['image_num'] = df_test.index + 200000

# Create the image_id column
def create_imageid(row):
    
    image_id = str(row['fine_labels']) + '_' + str(row['coarse_labels']) + '_' + str(row['image_num']) +'.jpg'
    
    return image_id

df_test['image_id'] = df_test.apply(create_imageid, axis=1)


# Create the fine and coarse label names columns

def create_finelabelname(x):
    
    # this returns bytes: b'apple'
    name = fine_label_names_list[x]
    
    # convert bytes to string: 'apple'
    name = name.decode("utf-8") 
    
    return name


def create_coarselabelname(x):
    
    # this returns bytes: b'apple'
    name = coarse_label_names_list[x]
    
    # convert bytes to string: 'apple'
    name = name.decode("utf-8") 
    
    return name


df_test['fine_label_names'] = df_test['fine_labels'].apply(create_finelabelname)

df_test['coarse_label_names'] = df_test['coarse_labels'].apply(create_coarselabelname)


# Remove unnecessary columns
df_test = df_test.drop('image_num', axis=1)

# Reorder the columns
cols = ['image_id', 'fine_label_names', 'fine_labels', 'coarse_label_names', 'coarse_labels']
df_test = df_test[cols]

df_test.head()

In [ ]:
#@title

df_train.to_csv('train.csv', index=False)
df_test.to_csv('test.csv', index=False)

In [ ]:
#@title

train_csv = pd.read_csv('/content/train.csv')
test_csv = pd.read_csv('/content/test.csv')

train_data = pd.DataFrame(train_csv)
test_data =  pd.DataFrame(test_csv)

#CLEAR

In [ ]:
#@title

path_train = '/content/drive/MyDrive/cifar-100-python/train'

train_dict = unpickle(path_train)
train_matrix = train_dict[b'data']

path_test = '/content/drive/MyDrive/cifar-100-python/test'

test_dict = unpickle(path_test)
test_matrix = test_dict[b'data']

In [ ]:
#@title

print(test_matrix.shape)
train_data.head()

In [ ]:
# Create people directory
if os.path.isdir('people_images') == True:
  dir = '/content/people_images'
  for f in os.listdir(dir):
      os.remove(os.path.join(dir, f))

if os.path.isdir('people_images') == False:
    train_images = 'people_images'
    os.mkdir(train_images)


# Create not_people directory
if os.path.isdir('not_people_images') == True:
  dir = '/content/not_people_images'
  for f in os.listdir(dir):
      os.remove(os.path.join(dir, f))

else:
    train_images = 'not_people_images'
    os.mkdir(train_images)

In [ ]:
#@title

#Combined dataframe of both test and train
combined_data = train_data.append(test_data)
combined_matrix = np.append(train_matrix,test_matrix)

In [ ]:
#@title

random = combined_data[(combined_data['fine_labels']==0)].sample(31,replace=False)        # Initialize random dataframe
# select 31 random images from each class(fine label)
for i in range(1,100):
  label_index = combined_data[(combined_data['fine_labels']==i)].sample(31,replace=False)
  random = random.append(label_index)

In [ ]:
#@title

# People dataframe from combined
people_data = combined_data[(combined_data['coarse_label_names']=='people')]

# Randomize people dataframe
people = people_data.sample(frac=1)
people.head()

In [ ]:
#@title

# Remove all the people elements 
not_people = random[random.coarse_label_names != 'people']

# add 55 random elements from not people to make 3000 images (equal to people)
not_people = not_people.append(not_people.sample(n=55))

#Randomize not_people
not_people = not_people.sample(frac=1)

#Convert and store not_people dataframe to csv
# not_people.to_csv('not_people.csv')
print(len(not_people))
not_people.head()

In [ ]:
#@title

count = 0
print(len(people))

In [ ]:
#@title

# Prepare train images
for i in range(0, train_matrix.shape[0]):
    if(train_data.loc[i,'image_id'] in people.values):
      
      # Get the image_id from the df_train dataframe
      image_id = train_data.loc[i, 'image_id']


      # Select an image
      row = train_matrix[i]

      # Extract each channel
      ch0 = row[0:1024] 
      ch1 = row[1024:2048]
      ch2 = row[2048:]

      # Reshape to 32x32
      ch0 = np.reshape(ch0, (32,32)) # red
      ch1 = np.reshape(ch1, (32,32)) # green
      ch2 = np.reshape(ch2, (32,32)) # blue

      # Stack the matrices along the channel axis
      image = np.dstack((ch0, ch1, ch2))

      
      # Save the image in the folder
      # that we created.
      fname = image_id
      dst = os.path.join('people_images', fname)
      
      # If cv2.COLOR_RGB2BGR is not used then the saved images appear blue.
      cv2.imwrite(dst, image)
      count = count + 1

# Not-People
    if(train_data.loc[i,'image_id'] in not_people.values):
      # Get the image_id from the df_train dataframe
      image_id = train_data.loc[i, 'image_id']


      # Select an image
      row = train_matrix[i]

      # Extract each channel
      ch0 = row[0:1024] 
      ch1 = row[1024:2048]
      ch2 = row[2048:]

      # Reshape to 32x32
      ch0 = np.reshape(ch0, (32,32)) # red
      ch1 = np.reshape(ch1, (32,32)) # green
      ch2 = np.reshape(ch2, (32,32)) # blue

      # Stack the matrices along the channel axis
      image = np.dstack((ch0, ch1, ch2))

      
      # Save the image in the folder
      # that we created.
      fname = image_id
      dst = os.path.join('not_people_images', fname)
      
      # If cv2.COLOR_RGB2BGR is not used then the saved images appear blue.
      cv2.imwrite(dst, image)
      count = count + 1

In [ ]:
#@title

# Prepare test images
for i in range(0, test_matrix.shape[0]):
    if(test_data.loc[i,'image_id'] in people.values):
      
      # Get the image_id from the df_train dataframe
      image_id = test_data.loc[i, 'image_id']


      # Select an image
      row = test_matrix[i]

      # Extract each channel
      ch0 = row[0:1024] 
      ch1 = row[1024:2048]
      ch2 = row[2048:]

      # Reshape to 32x32
      ch0 = np.reshape(ch0, (32,32)) # red
      ch1 = np.reshape(ch1, (32,32)) # green
      ch2 = np.reshape(ch2, (32,32)) # blue

      # Stack the matrices along the channel axis
      image = np.dstack((ch0, ch1, ch2))

      
      # Save the image in the folder
      # that we created.
      fname = image_id
      dst = os.path.join('people_images', fname)
      
      # If cv2.COLOR_RGB2BGR is not used then the saved images appear blue.
      cv2.imwrite(dst, image)
      count = count + 1

# Not-People
    if(test_data.loc[i,'image_id'] in not_people.values):
      # Get the image_id from the df_train dataframe
      image_id = test_data.loc[i, 'image_id']


      # Select an image
      row = test_matrix[i]

      # Extract each channel
      ch0 = row[0:1024] 
      ch1 = row[1024:2048]
      ch2 = row[2048:]

      # Reshape to 32x32
      ch0 = np.reshape(ch0, (32,32)) # red
      ch1 = np.reshape(ch1, (32,32)) # green
      ch2 = np.reshape(ch2, (32,32)) # blue

      # Stack the matrices along the channel axis
      image = np.dstack((ch0, ch1, ch2))

      
      # Save the image in the folder
      # that we created.
      fname = image_id
      dst = os.path.join('not_people_images', fname)
      
      # If cv2.COLOR_RGB2BGR is not used then the saved images appear blue.
      cv2.imwrite(dst, image)
      count = count + 1

In [ ]:
os.mkdir('dataset')
shutil.move('/content/not_people_images','dataset/')
shutil.move('/content/people_images','dataset/')

In [ ]:
#@title
# Convert and store people dataframe to csv
people.to_csv('people.csv')

In [ ]:
#@title

#!zip -r /content/people.zip /content/people_images
#from google.colab import files
#files.download("/content/people.zip")

#!zip -r /content/not_people.zip /content/not_people_images
#from google.colab import files
#files.download("/content/not_people.zip")

In [ ]:
#@title

count

In [ ]:
# IMPORT DATASET



BATCH_SIZE = 32
IMG_SIZE = (224, 224)
directory = "dataset/"
train_dataset = image_dataset_from_directory(directory,
                                             shuffle=True,
                                             image_size=IMG_SIZE,
                                             validation_split=0.2,
                                             subset='training',
                                             seed=42)
test_dataset = image_dataset_from_directory(directory,
                                             shuffle=True,
                                             image_size=IMG_SIZE,
                                             validation_split=0.2,
                                             subset='validation',
                                             seed=42)

In [ ]:
class_names = train_dataset.class_names
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
def data_augmenter():
    '''
    Create a Sequential model composed of 2 layers
    Returns:
        tf.keras.Sequential
    '''
    ### START CODE HERE
    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(RandomFlip("horizontal"))
    data_augmentation.add(RandomRotation(0.2))
    ### END CODE HERE
    
    return data_augmentation

In [ ]:
data_augmentation = data_augmenter()
data_augmentation = data_augmenter()
for image, _ in train_dataset.take(1):
    plt.figure(figsize=(10, 10))
    first_image = image[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
        plt.imshow(augmented_image[0] / 255)
        plt.axis('off')

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
IMG_SIZE = (224,224)
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(include_top=False,weights='imagenet')
base_model.summary()
type(base_model)


In [ ]:
# Get imagenet labels
# import tensorflow_datasets as tfds
# labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
# imagenet_labels = np.array(open(labels_path).read().splitlines())

# Set data_dir to a read-only storage of .tar files
# Set write_dir to a w/r storage
# data_dir = 'datasets/imagenet/'
# write_dir = 'psando/tf-imagenet-dirs'

# Construct a tf.data.Dataset
# download_config = tfds.download.DownloadConfig(
#                       extract_dir=os.path.join(write_dir, 'extracted'),
#                       manual_dir=data_dir
#                   )
# download_and_prepare_kwargs = {
#     'download_dir': os.path.join(write_dir, 'downloaded'),
#     'download_config': download_config,
# }
# ds = tfds.load('imagenet2012_subset', 
#                data_dir=os.path.join(write_dir, 'data'),         
#                split='train', 
#                shuffle_files=False, 
#                download=True, 
#                as_supervised=True,
#                download_and_prepare_kwargs=download_and_prepare_kwargs)

# Set data_dir to a read-only storage of .tar files
# Set write_dir to a w/r storage
# data_dir = '/content/image net dataset'
# write_dir = 'psando/tf-imagenet-dirs'



AssertionError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 32
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = 6000
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.30,
                                                               final_sparsity=0.90,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(base_model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

In [ ]:
# logdir = tempfile.mkdtemp()

# callbacks = [
#   tfmot.sparsity.keras.UpdatePruningStep(),
#   tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
# ]

# model_for_pruning.fit(train_dataset,
#                   batch_size=batch_size, epochs=epochs, validation_split=validation_split,
#                   callbacks=callbacks)

In [ ]:
# # Create a base model
# #base_model = tf.keras.applications.MobileNetV2(include_top=False,weights='imagenet')
# #base_model.load_weights('imagenet') # optional but recommended for model accuracy

# # Helper function uses `prune_low_magnitude` to make only the 
# # Dense layers train with pruning.
# def apply_pruning_to_dense(layer):
#   if isinstance(layer, tf.keras.layers.Dense):
#     return tfmot.sparsity.keras.prune_low_magnitude(layer)
#   return layer

# # Use `tf.keras.models.clone_model` to apply `apply_pruning_to_dense` 
# # to the layers of the model.
# model_for_pruning = tf.keras.models.clone_model(
#     base_model,
#     clone_function=apply_pruning_to_dense,
# )

# model_for_pruning.summary()


In [ ]:
image_batch, label_batch = next(iter(train_dataset))

#Resize 32x32 to 224x224
tf.image.resize(
    image_batch,
    (224,224),
    preserve_aspect_ratio=True)
# feature_batch = base_model(image_batch)
# print(feature_batch.shape)

In [ ]:
def people_model(base_model,image_shape=IMG_SIZE, data_augmentation=data_augmenter()):
    ''' Define a tf.keras model for binary classification out of the MobileNetV2 model
    Arguments:
        image_shape -- Image width and height
        data_augmentation -- data augmentation function
    Returns:
    Returns:
        tf.keras.model
    '''
    
    
    input_shape = image_shape + (3,)
    
    ### START CODE HERE
    
    # base_model = tf.keras.applications.MobileNetV2(input_shape= input_shape,
    #                                                include_top=False, # <== Important!!!!
    #                                                weights='imagenet') # From imageNet
    
    # freeze the base model by making it non trainable
    base_model.trainable = False

    # create the input layer (Same as the imageNetv2 input size)
    inputs = tf.keras.Input(shape=input_shape) 
    
    # apply data augmentation to the inputs
    x = data_augmentation(inputs)
    
    # data preprocessing using the same weights the model was trained on
    x = preprocess_input(x) 
    
    # set training to False to avoid keeping track of statistics in the batch norm layer
    x = base_model(x, training=False) 
    
    # add the new Binary classification layers
    # use global avg pooling to summarize the info in each channel
    x = tfl.GlobalAveragePooling2D()(x)  
    # include dropout with probability of 0.2 to avoid overfitting
    x = tfl.Dropout(0.2)(x)
        
    # use a prediction layer with one neuron (as a binary classifier only needs one)
    outputs = tfl.Dense(1)(x) 
    
    ### END CODE HERE
    
    model = tf.keras.Model(inputs, outputs)
    
    return model

In [ ]:
model2 = people_model(base_model,IMG_SIZE, data_augmentation)

In [ ]:
base_learning_rate = 0.003
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model2.summary()

In [ ]:
initial_epochs = 10
history = model2.fit(train_dataset, validation_data=test_dataset, epochs=initial_epochs)

In [ ]:
acc = [0.] + history.history['accuracy']
val_acc = [0.] + history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
#base_model = model2.layers[4]
#base_model.trainable = True
# Let's take a look to see how many layers are in the base model
# print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
#fine_tune_at = 120

### START CODE HERE

# Freeze all the layers before the `fine_tune_at` layer
# for layer in base_model.layers[:fine_tune_at]:
#     layer.trainable = True
    
# Define a BinaryCrossentropy loss function. Use from_logits=True
#loss_function=tf.keras.losses.BinaryCrossentropy(from_logits=True)
# Define an Adam optimizer with a learning rate of 0.1 * base_learning_rate
#optimizer = tf.keras.optimizers.Adam(learning_rate=0.1*base_learning_rate)
# Use accuracy as evaluation metric
#metrics=['accuracy']

### END CODE HERE

#model2.compile(loss=loss_function,
#              optimizer = optimizer,
 #             metrics=metrics)

In [ ]:
#fine_tune_epochs = 5
#total_epochs =  initial_epochs + fine_tune_epochs

#history_fine = model2.fit(train_dataset,
#                         epochs=total_epochs,
#                         initial_epoch=history.epoch[-1],
#                         validation_data=validation_dataset)

In [ ]:
#acc += history_fine.history['accuracy']
#val_acc += history_fine.history['val_accuracy']

#loss += history_fine.history['loss']
#val_loss += history_fine.history['val_loss']

In [ ]:
#plt.figure(figsize=(8, 8))
#plt.subplot(2, 1, 1)
#plt.plot(acc, label='Training Accuracy')
#plt.plot(val_acc, label='Validation Accuracy')
#plt.ylim([0, 1])
#plt.plot([initial_epochs-1,initial_epochs-1],
#          plt.ylim(), label='Start Fine Tuning')
#plt.legend(loc='lower right')
#plt.title('Training and Validation Accuracy')

#plt.subplot(2, 1, 2)
#plt.plot(loss, label='Training Loss')
#plt.plot(val_loss, label='Validation Loss')
#plt.ylim([0, 1.0])
#plt.plot([initial_epochs-1,initial_epochs-1],
#         plt.ylim(), label='Start Fine Tuning')
#plt.legend(loc='upper right')
#plt.title('Training and Validation Loss')
#plt.xlabel('epoch')
#plt.show()

In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, "/content/Saved_Model", include_optimizer=False)

In [ ]:
# converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
# pruned_tflite_model = converter.convert()

# _, tflite_file = tempfile.mkstemp('.tflite')

# with open("/content/tflite_file", 'wb') as f:
#   f.write(pruned_tflite_model)

In [ ]:
#def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
 # import os
 # import zipfile

 # _, zipped_file = tempfile.mkstemp('.zip')
 # with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  #  f.write(file)

 # return os.path.getsize(zipped_file)

In [ ]:
#print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
#print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

In [ ]:
# def representative_dataset():
#     for _ in range(100):
#       data = np.random.rand(1, 244, 244, 3)
#       yield [data.astype(np.float32)]

In [ ]:
import sys
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

_, quantized_tflite_file = tempfile.mkstemp('.tflite')

with open("/content/Saved_Model.tflite", 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)
print('Saved quantized and pruned TFLite model to:', quantized_tflite_file)
print("Size of tflite model ",sys.getsizeof(quantized_and_pruned_tflite_model))

In [ ]:
# import sys
# converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.representative_dataset = representative_dataset
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# converter.inference_input_type = tf.int8  # or tf.uint8
# converter.inference_output_type = tf.int8  # or tf.uint8
# quantized_and_pruned_tflite_model = converter.convert()

# _, quantized_tflite_file = tempfile.mkstemp('.tflite')

# with open("/content/quantized_tflite_file", 'wb') as f:
#   f.write(quantized_and_pruned_tflite_model)
# print('Saved quantized and pruned TFLite model to:', quantized_tflite_file)
# print("Size of tflite model ",sys.getsizeof(quantized_and_pruned_tflite_model))

In [ ]:
!apt-get update && apt-get -qq install xxd
!xxd -i "/content/Saved_Model.tflite" > model_data.cc